# Exercises B: Solutions
_Version: October 31, 2020, see_ [PyEcon.org](https://pyecon.org).

### Exercise 1: NHL Hockey
The file `nhl1617.txt` contains data of all (non-goalie) players in the NHL who had at least some _ice time_, i.e. playing time, in the 2016/2017 regular season - without play-offs. Load the CSV data of the file `nhl1617.txt` in Python using `open` and further string transformations (_see below_). Try to cope with the huge string and allocate the information to a dictionary. Use the column names of the dataset as keys for the dictionary.

Solve these tasks using only a dictionary consisting of lists but **not using** _pandas_:

1. How many players have been recorded?
1. A column _points_ ("P") is missing in the data. The number of points of a player is defined as the sum of his _goals_ ("G") and _assists_ ("A"). Add the points to your dictionary.
1. Who is the top scorer in terms of points?
1. How many German (non-goalie) players had some ice time in there 2016/2017 regular season? _Hint: Nationality of a player can be found in the dictionary entry "Nat". Germans are indicated by "DEU"._
1. What are their names? *Hint: Names can be found in th entry "Last_Name" and "First_Name".*
1. Who performed best among the German players in terms of points ("P")?
1. How many points ("P") did he have?
1. How well did he perform in the entire league? Put differently, what was his rank in terms of points?
1. Find the top ten scorers (in terms of points) and print them including their number of point and their respective team.
1. What are the three countries with the most players originating from?

If you feel comfortable looping through dictionaries and lists, re-implement your codes **using** _pandas_.

In [ ]:
with open("data/nhl1617.txt") as f:
    data = f.read()
data = data.replace('"', '')
data = data.splitlines()
var_names = data[0]
var_names = var_names.rsplit(";")
data.pop(0)
NHL = {}
for keys in var_names:
    NHL[keys] = []
for line in data:
    line = line.rsplit(';')
    for info, keys in zip(line, NHL):
        NHL[keys].append(info)
NHL["Last_Name"][:10]

In [ ]:
# Total number of players
n = len(NHL["Last_Name"])
n

In [ ]:
# Creating the points column
NHL["P"] = []
for G, A in zip(NHL["G"], NHL["A"]):
    NHL["P"].append(int(G) + int(A))
NHL["P"][:10]

In [ ]:
# Finding the top scorer(s)
max_p = max(NHL["P"])
index = [i for i in range(n) if NHL["P"][i] == max_p]
if len(index) == 1:
    s = " is"
else:
    s = "s are"
print(f"The top scorer{s}:")
for i in index:
    print(f"{NHL['Last_Name'][i]}, {NHL['First_Name'][i]}")

In [ ]:
# Finding the German (non-goalie) players of the regular season
index = [i for i in range(n) if NHL["Nat"][i] == "DEU"]

# Alternative
# count = 0
# for nation in NHL["Nat"]:
#     if nation == "DEU":
#         count = count + 1

print(f"In the 2016/17 regular season there were {len(index)}"
      f" German (non-goalie) players:")
for i in index:
    print(f"{NHL['Last_Name'][i]}, {NHL['First_Name'][i]}")

In [ ]:
# Finding the German top scorer(s)
ger_p = [NHL["P"][i] for i in index]
max_p = max(ger_p)
index_p = [index[i] for i, p in enumerate(ger_p) if p == max_p]
for i in index_p:
    print(f"{NHL['Last_Name'][i]} (Points: {max_p}) was"
          f" the best performing German NHL player.")

In [ ]:
# Assessing relative performance
count = 1
for i in NHL["P"]:
    if i > max_p:
        count += 1
print(f"{NHL['Last_Name'][index_p[0]]} was the {count}th"
      f" best performer in the entire League with respect to points.")

In [ ]:
# Top 10 scorers
index = [i for i, j in sorted(enumerate(NHL["P"]),
                              reverse=True,
                              key=lambda x: x[1])]
for i in range(0, 10):
    print(f"{i + 1:2}. {NHL['Last_Name'][index[i]]:15}"
          f"  Team: {NHL['Team'][index[i]]:10}"
          f"  Points: {NHL['P'][index[i]]:3}")

In [ ]:
# The three countries with the most players originating from.
counts = {}
for nation in NHL["Nat"]:
    if nation in counts:
        counts[nation] += 1
    else:
        counts[nation] = 1
top3 = sorted(counts.items(), key=lambda x: x[1], reverse=True)[0:3]
for nation, count in top3:
    print(f"{nation}: {count}")

#### Solution using pandas:

In [ ]:
import pandas as pd
nhl = pd.read_csv("data/nhl1617.txt", sep=";")
nhl.count()

In [ ]:
nhl["P"] = nhl["G"] + nhl["A"]
nhl.head()

In [ ]:
max_index = nhl["P"].idxmax()
nhl.iloc[max_index]

In [ ]:
germans = nhl[nhl["Cntry"] == "DEU"]
print(germans.count())
germans

In [ ]:
max_index = germans["P"].idxmax()
nhl.iloc[max_index]

In [ ]:
nhl = nhl.sort_values('P', ascending=False).reset_index()
nhl[nhl["Last_Name"] == "Draisaitl"]


In [ ]:
nhl.iloc[0:10]

In [ ]:
country = nhl[["Cntry", "Last_Name"]].groupby(["Cntry"]).count()\
          .sort_values("Last_Name", ascending=False)
country[:3]

### Exercise 2: Same sex marriage - "Ehe für Alle"
The file `data/votings.txt` contains the voting behavior of all members of the german parliament regarding a draft that aimed to legalise same sex marriage. Party membership is also part of the dataset.

The following list contains the ministers: ['Merkel', 'Altmaier', 'Friedrich, Dr.', 'Leyen', 'Schmidt, Christian', 'Dobrindt', 'Maizière, Dr.', 'Schäuble']

Note that _Wanka_ was not part of the German parliament. _Gröhe_ and _Müller_ did not participate in the poll.

1. We are interested in the share of voting in favor of same sex marriage grouped by party membership.
1. Display the votings of the members of the CDU/CSU who were also part of the top-level administration.

In [ ]:
names = ['Merkel', 'Altmaier', 'Friedrich, Dr.', 'Leyen', 'Schmidt, Christian',
         'Dobrindt', 'Maizière, Dr.', 'Schäuble']

In [ ]:
# one solution using pandas
import pandas as pd

df = pd.read_csv("data/votings.txt", sep=";")
df.head()

In [ ]:
res = df.groupby(["Party", "Vote"]).count()
res.columns = ["abs"]
res

In [ ]:
res["rel"] = res / res.groupby(level=0).sum()
res

In [ ]:
# without counting the non-votes
res = df[df.Vote != "Nicht abg."].groupby(["Party", "Vote"]).count()
res.columns = ["abs"]
res["rel"] = res / res.groupby(level=0).sum()
res

In [ ]:
# yet another solution without using pandas
with open("data/votings.txt") as f:
    data = f.read()
data = data.replace('"', '')
data = data.splitlines()
var_names = data[0]
var_names = var_names.rsplit(';')
data.pop(0)
vote = {}
for keys in var_names:
    vote[keys] = []
for line in data:
    line = line.rsplit(';')
    for info, keys in zip(line, vote):
        vote[keys].append(info)
parties = {}
for party in list(set(vote["Party"])):
    parties[party] = []
for party in parties:
    num_assembly = [1 for i, j in zip(vote["Vote"], vote["Party"])
                    if i != "Nicht abg." and j == party]
    num_assembly = sum(num_assembly)
    share = [1 for i, j in zip(vote["Vote"], vote["Party"])
             if i == "Ja" and j == party]
    share = sum(share) / num_assembly
    parties[party] = share
print(parties)

In [ ]:
# one solution using pandas
df[df.Assemblyman.str.contains("|".join(names))]

In [ ]:
# yet another solution without using pandas
index = []

for i in range(0, len(vote['Assemblyman'])):
    for j in names:
        if j in vote['Assemblyman'][i]:
            index.append(i)

for i in index:
    assemblyman = vote['Assemblyman'][i]
    name = assemblyman[assemblyman.find(',')+1:].replace(' ', '')
    name = name + ' ' + assemblyman[0:assemblyman.find(',')]
    print(name, 'voted "%s".' % vote['Vote'][i])

### Exercise 3: Stock market data
1. Read the file `amzn.csv` into a DataFrame.
1. Read the file `dji.csv` into a new DataFrame.
1. Remove all columns in both DataFrames except _Adj Close_ and _Volume_.
1. Rename columns to _Amzn Adj Close_, _Amzn Volume_, _Dji Adj Close_, _Dji Volume_.

In [ ]:
import pandas as pd
amzn = pd.read_csv("data/amzn.csv", index_col=0, parse_dates=True)
dow = pd.read_csv("data/dji.csv", index_col=0, parse_dates=True)
amzn = amzn.drop(["Open", "Close", "High", "Low"], axis=1)
dow = dow.drop(["Open", "Close", "High", "Low"], axis=1)
amzn = amzn.rename(columns={"Adj Close": "Amzn Adj Close",
                            "Volume": "Amzn Volume"})
dow = dow.rename(columns={"Adj Close": "Dji Adj Close",
                          "Volume": "Dji Volume"})
amzn.head()

In [ ]:
dow.head()

### Exercise 4: Performances
1. Concatenate both DataFrames along the _Date_ (rows) axis.
1. Remove all rows which contain no data for Amazon.
1. Let _Adj Close_ (2017-02-23) be the reference price (the base). Add two columns which give the _Adj Close_ as a percentage of the reference price.
1. Add a column that contains the 10-days-rolling-correlation between the _Adj Close_ of Amazon and the Dji.
1. Calculate the mean of the correlation columns.

In [ ]:
all_data = pd.concat([amzn, dow], axis=1)
all_data = all_data.dropna(axis=0, how="any")
all_data["Amzn percentage"] = all_data["Amzn Adj Close"] /\
                              all_data["Amzn Adj Close"][0] * 100
all_data["Dji percentage"] = all_data["Dji Adj Close"] /\
                             all_data["Dji Adj Close"][0] * 100
all_data["Corr"] = all_data["Amzn Adj Close"].rolling(window=10).\
                                              corr(all_data["Dji Adj Close"])
print(all_data["Corr"].mean())

### Exercise 5: Simple trading algorithm
Consider the Amazon DataFrame.

1. Add two columns containing the upper and lower Bollinger Band (10-days-rolling-mean +/- standard deviation of the 10-days-rolling-mean).
1. Add a column which says `BUY` if the _Adj Close_ breaks through the lower band (coming from below the lower band) and says `SELL` if the _Adj Close_ falls below the upper band (coming from above the upper band).
1. Write a function that you provide with an initial investment and that computes the total return of investment based on the signals `BUY` and `SELL`.
    1. Specify the amount of money to invest at the beginning.
    1. At every `BUY` signal, you spend the total amount of your money to buy Amazon stocks.
    1. At every `SELL` signal, you sell all stocks again.
    1. At the end of the full period, compute the total return.

In [ ]:
amzn["Rolling"] = amzn["Amzn Adj Close"].rolling(window=10).mean()
amzn["Std"] = amzn["Amzn Adj Close"].rolling(window=10).std()
amzn["Upper"] = amzn["Rolling"] + amzn["Std"]
amzn["Lower"] = amzn["Rolling"] - amzn["Std"]
amzn["Trade"] = ""

%matplotlib inline
amzn[["Lower", "Amzn Adj Close", "Upper"]].plot(figsize=(12, 12))

In [ ]:
for i in range(1, len(amzn.index)):
    if amzn["Amzn Adj Close"][i] > amzn["Lower"][i] and\
       amzn["Amzn Adj Close"][i-1] < amzn["Lower"][i-1]:
        amzn.at[amzn.index[i], 'Trade'] = 'BUY'

    if amzn["Amzn Adj Close"][i] < amzn["Upper"][i] and\
       amzn["Amzn Adj Close"][i-1] > amzn["Upper"][i-1]:
        amzn.at[amzn.index[i], 'Trade'] = 'SELL'

amzn.head(20)

In [ ]:
amzn.tail(20)

In [ ]:
def algo(amzn, invest, lag=1):
    last_order = "SELL"
    stocks = 0
    cash = invest
    for i in range(lag, len(amzn.index)):
        if amzn["Trade"][i-lag] == "BUY" and last_order == "SELL":
            stocks = cash / amzn["Amzn Adj Close"][i]
            cash = 0
            last_order = "BUY"
            print(f"Buy {stocks:5.2f} stocks on {amzn.index[i].date()}"
                  f" at a price of {amzn['Amzn Adj Close'][i]:7.2f}")
        if amzn["Trade"][i-lag] == "SELL" and last_order == "BUY":
            print(f"Sell {stocks:4.2f} stocks on {amzn.index[i].date()}"
                  f" at a price of {amzn['Amzn Adj Close'][i]:7.2f}")
            cash = amzn["Amzn Adj Close"][i] * stocks
            stocks = 0
            last_order = "SELL"
    if stocks > 0:
        cash = amzn["Amzn Adj Close"][-1] * stocks
    return (cash - invest) / invest * 100


# let's invest 1000 USD
invest = 1000
total_return = algo(amzn, invest)
print(f"Your total return of investment is {total_return:.2f} %,"
      f" i.e. {total_return * invest / 100:.2f} USD!")